## Load libraries

In [1]:
gpu=0
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=$gpu
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, '/home/kevin/fastai')
from fastai.structured import *
from fastai.column_data import *
from IPython.display import HTML
np.set_printoptions(threshold=50, edgeitems=20)
from datetime import timedelta
#paths
data_path = '/home/kevin/nbs/kaggle/GrocerySales/data/'
slowdata = data_path + 'slowdisk/'; slowdata
simple = slowdata + 'simple_fastai/'

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


## Functions

In [74]:
#normalize and denormalize targets
def normy(y):
    meany=np.mean(y)
    stdevy = np.std(y)
    ynorm = (y-meany)/stdevy
    return meany, stdevy, ynorm

def denorm(y, meany, stdevy, r=3):
    return np.round(y*stdevy+meany, r)

#custom function I wrote to add a null check
def apply_cats(df, trn, null_check=False):
    for n,c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name=='category'):
            df[n] = pd.Categorical(c, categories=trn[n].cat.categories, ordered=True)
            if null_check and df[n].isnull().values.any():
                raise ValueError(f'Target dataframe has null values for column {n}. This can occur if the target dataframe has category values not present in original.')

### Load data
See [this notebook](https://github.com/kevindewalt/ai-notebooks/blob/master/Corporaci%C3%B3n%2BFavorita%2BGrocery%2BSales%2BForecasting.ipynb).

In [3]:
train = pd.read_pickle(simple + 'train_before_fastai.pkl')
test = pd.read_pickle(simple + 'test_before_fastai.pkl')

In [76]:
train.head()

,store_nbr,item_nbr,unit_sales,onpromotion,m_ratio
date,,,,,
2017-03-01,1,105574,10.000001,0,5.825894
2017-03-01,1,105575,18.000000,0,10.596720
2017-03-01,1,105737,3.000000,0,1.167335
2017-03-01,1,106716,2.000000,0,2.344336
2017-03-01,1,108698,6.000000,0,1.047796


In [77]:
test.head()

,store_nbr,item_nbr,onpromotion,m_ratio
date,,,,
2017-08-16,1,96995,0,0.123018
2017-08-16,1,99197,0,0.269043
2017-08-16,1,103501,0,0.000000
2017-08-16,1,103520,0,0.883692
2017-08-16,1,103665,0,1.925161


## Prep cat and cont vars

In [4]:
#use the last 10% of data for validation.
train_ratio = 0.90
train_size = int(train_ratio*len(train))
val_idx = list(range(train_size, len(train)))

#have to merge train & test since some cats in test are not in train.
cat_vars = ['item_nbr', 'onpromotion', 'store_nbr']
merged = pd.concat([train[cat_vars],test[cat_vars]])
for v in cat_vars: merged[v] = merged[v].astype('category').cat.as_ordered()

apply_cats(train, merged, null_check=True)
apply_cats(test, merged, null_check=True)

contin_vars = ['m_ratio']
for v in contin_vars:
    train[v] = train[v].astype('float32')
    test[v] = test[v].astype('float32')

## Generate model dataframes

In [5]:
train = train.set_index('date')
df, y, nas, mapper = proc_df(train, 'unit_sales', do_scale=True)

test = test.set_index('date')
test['ignore']=0 #need a dummy column to work with proc_df
df_test, _, nas, mapper = proc_df(test, 'ignore', do_scale=True, mapper=mapper, na_dict=nas)

test.drop('ignore', 1, inplace=True)

In [13]:
y_n = normy(y)
yl=y_n[2] 
len(yl)==len(df)

True

In [36]:
n_cont = len(df.columns)-len(cat_vars)

max_log_y = np.max(yl)
min_log_y = np.min(yl)
y_range = (min_log_y*1.2, max_log_y*1.2)

## Attempt to replicate previous Keras results 

In [65]:
emb_szs = [(4060,5), (3,3), (55,3)]

In [66]:
# del md; del m

In [67]:
bs = int(1e5)
md = ColumnarModelData.from_data_frame(path=simple, val_idxs=val_idx, df=df, y=yl,
                                      cat_flds=cat_vars, bs=bs, test_df=df_test)

In [68]:
m = md.get_learner(emb_szs=emb_szs, n_cont=n_cont, emb_drop=0.00, 
                   out_sz=1, szs=[10,5], drops=[0.0,0.0], y_range=y_range)

In [69]:
#count trainable parameters. Results are similar to Keras model.
model_parameters = filter(lambda p: p.requires_grad, m.model.parameters())
sum([np.prod(p.size()) for p in model_parameters])

20697

In [62]:
#Custom loss function since I used 'mean_absolute_error' in Keras
def mae(inp, targ):
    l = len(inp)
    return torch.sum(torch.abs(inp-targ))/l

In [70]:
lr=1e-1
m.crit = mae

In [71]:
m.fit(lr,1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 0/327 [00:00<?, ?it/s]                     


Exception in thread Thread-14:
Traceback (most recent call last):
  File "/home/kevin/anaconda2/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/kevin/anaconda2/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/kevin/anaconda2/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



epoch      trn_loss   val_loss                               
    0      0.285734   0.262615  



[0.26261467]

In [72]:
m.fit(1e-2,1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                               
    0      0.26573    0.262615  



[0.26261467]

In [73]:
m.fit(1e-3,1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                               
    0      0.265203   0.262615  



[0.26261467]

In [79]:
m.fit(1e-1,2, cycle_len=2, cycle_mult=3)

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

  0%|          | 0/327 [00:00<?, ?it/s]                     


Exception in thread Thread-20:
Traceback (most recent call last):
  File "/home/kevin/anaconda2/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/kevin/anaconda2/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/kevin/anaconda2/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



epoch      trn_loss   val_loss                               
    0      0.265733   0.262615  
    1      0.265337   0.262615                               
    2      0.265391   0.262615                               
    3      0.265388   0.262615                               
    4      0.26559    0.262615                               
    5      0.265796   0.262615                               
    6      0.265813   0.262615                               
    7      0.265253   0.262615                               



[0.26261467]

# Observations

#### Fastai is much easier to use than Keras.
I had to write much less code to prep for training.

#### Unfortunately my loss in the fastai model is about 2x worse than what I achieved in Keras. 
Here is what I checked.
- Feature engineering is the same. I ran the same python code as previously to get the 3 category and 1 continuous features.
+ The models have the same number of trainable parameters.
+ The models have the same loss function, MAE.
+ I normalized both targets using the same function.

#### Training the fastai model is 10-20x slower than Keras.
I think Keras is copying the entire dataset to the GPU and then doing the training. Since I'm using gigantic batch sizes it runs very fast in Keras, about 22 secs. Takes >5 mins in fastai.